# Analyzing Work Hour Data

----

#### Summary
In this notebook I will be working on 'work_hour.csv'. This dataset contains work hour data for every year from 2000-2019. It shows how many hours of work people do per year (cumulative). I am interested in whether countries where people work for longer hours will have higher suicide rates, as pressure from work and overtime work is related to committing suicide. 

#### Key Question
* Is there a relationship between work hours and suicide rates? 

In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

df = pd.read_csv('../data/Raw_data/work_hour.csv')
suicide = pd.read_csv('../data/Cleaned_data/suicide_total.csv')

In [75]:
df

,COUNTRY,Country,EMPSTAT,Employment status,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,TE,Total employment,A,Annual,2000,2000,HOUR,Hours,0,Units,NaN,NaN,1831.0,NaN,NaN
1,AUS,Australia,TE,Total employment,A,Annual,2001,2001,HOUR,Hours,0,Units,NaN,NaN,1814.0,NaN,NaN
2,AUS,Australia,TE,Total employment,A,Annual,2002,2002,HOUR,Hours,0,Units,NaN,NaN,1794.0,NaN,NaN
3,AUS,Australia,TE,Total employment,A,Annual,2003,2003,HOUR,Hours,0,Units,NaN,NaN,1798.0,NaN,NaN
4,AUS,Australia,TE,Total employment,A,Annual,2004,2004,HOUR,Hours,0,Units,NaN,NaN,1801.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,ISR,Israel,DE,Dependent employment,A,Annual,2014,2014,HOUR,Hours,0,Units,NaN,NaN,1901.0,NaN,NaN
1469,ISR,Israel,DE,Dependent employment,A,Annual,2015,2015,HOUR,Hours,0,Units,NaN,NaN,1900.0,NaN,NaN
1470,ISR,Israel,DE,Dependent employment,A,Annual,2016,2016,HOUR,Hours,0,Units,NaN,NaN,1923.0,NaN,NaN
1471,ISR,Israel,DE,Dependent employment,A,Annual,2017,2017,HOUR,Hours,0,Units,NaN,NaN,1924.0,NaN,NaN


#### Cleaning up data
* I only want values from years 2000,2010,and 2016 (to match with suicide data), so I dropped the rows that are not from these years
* Dropped all columns except for country, year, and the work hours value
* Replaced the name of a country with it's more common name
* Because the data is long, which doesn't work well with a regression model, I used the pivot function to change it to a wide format
* Added a column which takes the mean value of the 3 years
* Lastly, merged with suicide data for further analysis

In [77]:
df.head(25)
work =df.loc[df['Time'].isin(['2000','2010','2016' ])]
work=work.filter(["COUNTRY", "Country","Time","Value"], axis=1)
work['Country']=work['Country'].replace(["Russian Federation"],'Russia')
work

,COUNTRY,Country,Time,Value
0,AUS,Australia,2000,1831.0
10,AUS,Australia,2010,1755.0
16,AUS,Australia,2016,1740.0
20,AUT,Austria,2000,1542.0
30,AUT,Austria,2010,1468.0
...,...,...,...,...
1445,CHE,Switzerland,2010,1597.0
1451,CHE,Switzerland,2016,1581.0
1454,ISR,Israel,2000,2022.0
1464,ISR,Israel,2010,1950.0


In [78]:
#from long to wide
work_wide=work.pivot_table(index="Country",columns="Time",values="Value")
work_wide

Time,2000,2010,2016
Country,,,
Australia,1819.00,1751.50,1741.00
Austria,1608.50,1510.00,1469.00
Belgium,1524.00,1503.00,1504.50
Canada,1782.50,1716.50,1710.00
Chile,2290.50,2096.50,2011.50
Costa Rica,2362.00,2265.80,2224.15
Czech Republic,1957.00,1841.50,1806.00
Denmark,1443.50,1410.50,1401.00
Estonia,1860.00,1775.00,1757.00


In [79]:
#add a column with the average value of 3 years
work_wide['means']=(work_wide[2016]+work_wide[2010]+work_wide[2000])/3
work_wide

Time,2000,2010,2016,means
Country,,,,
Australia,1819.00,1751.50,1741.00,1770.500000
Austria,1608.50,1510.00,1469.00,1529.166667
Belgium,1524.00,1503.00,1504.50,1510.500000
Canada,1782.50,1716.50,1710.00,1736.333333
Chile,2290.50,2096.50,2011.50,2132.833333
Costa Rica,2362.00,2265.80,2224.15,2283.983333
Czech Republic,1957.00,1841.50,1806.00,1868.166667
Denmark,1443.50,1410.50,1401.00,1418.333333
Estonia,1860.00,1775.00,1757.00,1797.333333


In [80]:
merged = pd.merge(left=work_wide, right=suicide, left_on='Country', right_on='Country')
merged.head()

,Country,2000,2010,2016,means_x,Sex,2016,2015,2010,2000,means_y
0,Australia,1819.00,1751.50,1741.00,1770.500000,Both sexes,11.7,12.2,11.0,12.2,11.633333
1,Austria,1608.50,1510.00,1469.00,1529.166667,Both sexes,11.4,11.5,12.2,16.0,13.200000
2,Belgium,1524.00,1503.00,1504.50,1510.500000,Both sexes,15.7,15.2,16.8,18.8,17.100000
3,Canada,1782.50,1716.50,1710.00,1736.333333,Both sexes,10.4,10.5,11.1,11.6,11.033333
4,Chile,2290.50,2096.50,2011.50,2132.833333,Both sexes,9.7,10.0,11.6,11.1,10.800000
5,Costa Rica,2362.00,2265.80,2224.15,2283.983333,Both sexes,7.5,7.5,8.4,7.7,7.866667
6,Denmark,1443.50,1410.50,1401.00,1418.333333,Both sexes,9.2,8.6,9.3,12.7,10.400000
7,Estonia,1860.00,1775.00,1757.00,1797.333333,Both sexes,14.4,13.3,14.9,25.4,18.233333
8,Finland,1610.50,1553.00,1530.00,1564.500000,Both sexes,13.8,12.4,16.6,21.6,17.333333
9,France,1501.00,1489.50,1475.00,1488.500000,Both sexes,12.1,12.5,14.8,16.9,14.600000


#### Examine relationships
I regressed work hours on suicide rate for each of the 3 years as well as the mean. The relationship is minimal if not non-existent for all 3 years and the mean, and the models fit no better than just the mean. This is somehow surprising given that the pressure from work is said to be related to suicide attempts. However there are some possible explanations for the absence of a relationship:
1. Length of work hours does not represent pressure from work
2. The sample size is small
3. Most importantly, I was not able to find data for low and mid income countries. If data for those countries are available I suspect that it will change the result significantly

In [81]:
m1 = smf.ols(formula= 'means_y ~ means_x', data=merged)
res = m1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                means_y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.029
Method:                 Least Squares   F-statistic:                   0.08181
Date:                Wed, 23 Dec 2020   Prob (F-statistic):              0.777
Time:                        02:39:33   Log-Likelihood:                -114.62
No. Observations:                  34   AIC:                             233.2
Df Residuals:                      32   BIC:                             236.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.9933      9.593      1.667      0.1

In [70]:
m1 = smf.ols(formula= 'Q("2016") ~ Q(2016)', data=merged)
res = m1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              Q("2016")   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                    0.1473
Date:                Sun, 20 Dec 2020   Prob (F-statistic):              0.704
Time:                        16:07:03   Log-Likelihood:                -101.51
No. Observations:                  34   AIC:                             207.0
Df Residuals:                      32   BIC:                             210.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.0981      6.624      2.128      0.0

In [71]:
m1 = smf.ols(formula= 'Q("2010") ~ Q(2010)', data=merged)
res = m1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              Q("2010")   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                 0.0003088
Date:                Sun, 20 Dec 2020   Prob (F-statistic):              0.986
Time:                        16:07:31   Log-Likelihood:                -114.96
No. Observations:                  35   AIC:                             233.9
Df Residuals:                      33   BIC:                             237.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.7032      8.707      1.459      0.1

In [72]:
m1 = smf.ols(formula= 'Q("2016") ~ Q(2016)', data=merged)
res = m1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              Q("2016")   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                    0.1473
Date:                Sun, 20 Dec 2020   Prob (F-statistic):              0.704
Time:                        16:07:55   Log-Likelihood:                -101.51
No. Observations:                  34   AIC:                             207.0
Df Residuals:                      32   BIC:                             210.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.0981      6.624      2.128      0.0